In [156]:
import pandas as pd
import numpy as np

In [157]:
from yahooquery import Ticker
def get_data(ticker, start = 0, end = 0, period = -1):
    stock = Ticker(ticker)
    if period == 'max':
        df = stock.history(period='max')
        
    else:
        df = stock.history(start=start, end=end)
    
    df = df.xs(ticker)
    return df

# Creating a portfolio

In [158]:
import random

In [159]:
ibov = pd.read_csv('IBOVDia_03-08-21 (1).csv')

In [160]:
stocks = ibov.iloc[:,0].values

In [161]:
def create_portfolio():
    portfolio_size = random.randint(3,10)#number of stocks in this portfolio

    portfolio_stocks = random.sample(list(stocks), portfolio_size) #portfolio_composition

    weights = np.random.random(portfolio_size)
    weights /= weights.sum() #portfolio weights

    df = get_data(portfolio_stocks[0]+'.SA', start='2020-03-03', end='2021-07-31')[['close']] * weights[0] #creating the dataframe
    for (stock, weight) in zip(portfolio_stocks[1:], weights[1:]):
        data = get_data(stock+'.SA', start='2020-03-03', end='2021-07-31')[['close']]* weight

        df['close'] += data['close']
    

    return (df, portfolio_stocks, weights)

In [162]:
(df, composition, weights) = create_portfolio()

# Metrics
- VAR
- RSI
- 252 SMA & 25 SMA

In [163]:
def var_historic(df, level=1):
    """
    Takes in a series of returns (r), and the percentage level
(level)
    Returns the historic Value at Risk at a specified level
    i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100-level) percent are above
    """
    r = df['close'].diff()

    r = r.dropna()
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        
        return -np.percentile(r, level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")

In [164]:
def rsi(df, periods = 14, ema = True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
        # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return list(rsi)[-1]

In [165]:
def sma(df):
    sma1 = df['close'].rolling(window=42).mean()
    last1 = list(sma1)[-1]
    sma2 = df['close'].rolling(window=252).mean()
    last2 = list(sma2)[-1]
    return (last1, last2)

In [173]:
portfolios = pd.DataFrame([[composition, sma(df)[0], sma(df)[1], rsi(df), var_historic(df)]])

In [174]:
for i in range(0, 25, 1):
    (df, composition, weights) = create_portfolio()
    portfolios = portfolios.append(pd.DataFrame([[composition, sma(df)[0], sma(df)[1], rsi(df), var_historic(df)]]))

In [177]:
portfolios.reset_index(drop=True, inplace=True)

In [179]:
portfolios.columns = ['stocks', 'SMA42', 'SMA252', 'RSI', 'VAR']

In [180]:
portfolios.to_csv('portfolios.csv')

In [181]:
portfolios

,stocks,SMA42,SMA252,RSI,VAR
0,"[YDUQ3, CSNA3, ECOR3, SUZB3]",37.535709,32.262438,41.945389,1.933511
1,"[PETR3, AZUL4, BBAS3, LWSA3, RENT3, WEGE3, MUL...",37.080673,35.202241,41.123368,2.427751
2,"[GNDI3, IRBR3, BRAP4, EQTL3, COGN3, JHSF3]",47.903355,42.769113,40.294641,2.778415
3,"[EZTC3, SANB11, BRFS3, BBDC4, HYPE3, QUAL3, EG...",30.365874,29.661823,35.028940,2.538826
4,"[LCAM3, JHSF3, ELET6, BRML3, QUAL3, ITUB4, CSA...",23.838953,21.551476,42.234431,1.824706
5,"[VIVT3, CYRE3, HGTX3, COGN3, JHSF3, HYPE3, BBD...",19.899637,17.348089,42.125579,1.398517
6,"[PETR4, ASAI3, RENT3, ENEV3, ITSA4, MRVE3, ABE...",36.204742,NaN,31.496022,1.439686
7,"[CCRO3, BBDC3, RADL3, WEGE3]",23.896189,23.418009,45.020413,1.251510
8,"[CPFE3, MRFG3, ELET6, CPLE6, ELET3, LCAM3]",25.506537,22.079824,36.397723,1.921796
9,"[EZTC3, ENGI11, BRDT3, YDUQ3]",35.902534,35.296362,26.235766,3.665750
